# Extended Source Analysis

The analysis of an extended source is performed similarly to the process described in the [binned likelihood](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/binned_likelihood_tutorial.html) tutorial.

The only extra step needed is to create a two dimensional template that describes the object you are wanting to analyze.

This template can be created from several different sources including maps from other observatories or you can create your own from scratch.

There are a few simple rules you have to follow in creating your template but once you've done that, pretty much any shape is possible to model.

***

This sample analysis is based on the Centaurus A analysis performed by the LAT team and described in [Abdo, A. A. et al. 2010, Science, 328, 728](http://adsabs.harvard.edu/abs/2010Sci...328..725F). At certain points we will refer to this article and its supplement as well as the [Cicerone](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/).

The goal of this tutorial is to reproduce the data analysis performed in this publication including calculating the spectral shape and fluxes of the central core of Cen A and the large radio lobes. This tutorial uses a [user-contributed tool](https://github.com/physicsranger/make4FGLxml) (`make4FGLxml.py`) and assumes you have the most recent [Fermitools](https://fermi.gsfc.nasa.gov/ssc/data/analysis/software/) installed.

We will also make significant use of python, so you might want to familiarize yourself with python (there's a beginner's guide at http://wiki.python.org/moin/BeginnersGuide).

This tutorial also assumes that you've gone through the non-python based [binned likelihood](https://github.com/fermi-lat/AnalysisThreads/blob/master/SourceAnalysis/1.BinnedLikelihood/binned_likelihood_tutorial.ipynb) thread. It is also useful if you have gone through the [Likelihood Analysis with Python](https://github.com/fermi-lat/AnalysisThreads/blob/master/SourceAnalysis/3.PythonLikelihood/python_tutorial.ipynb) tutorial.

This tutorial should take approximately 8 hours to complete (depending on your computer's speed) if you do everything, but there are some steps you can skip along the way which shaves off about 4 hours of that.

# Get the data

For this thread the original data were extracted from the [LAT data server](https://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi) with the following selections (these selections are similar to those in the paper):

```
Search Center (RA,Dec) = (201.47,-42.97)
Radius = 30 degrees
Start Time (MET) = 239557420 seconds (2008-08-04T15:43:40)
Stop Time (MET) = 265507200 seconds (2009-06-01T00:00:00)
Minimum Energy = 300 MeV
Maximum Energy = 300000 MeV
```


These are the event files. Run the code cell below to retrieve them:
```
L1504211512544B65347F11_PH00.fits
L1504211512544B65347F11_PH01.fits
L1504211512544B65347F11_PH02.fits
L1504211512544B65347F11_SC00.fits
```

In [1]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/L1504211512544B65347F11_PH00.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/L1504211512544B65347F11_PH01.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/L1504211512544B65347F11_PH02.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/L1504211512544B65347F11_SC00.fits

--2025-09-15 11:12:07--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/L1504211512544B65347F11_PH00.fits
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31608000 (30M) [application/fits]
Saving to: ‘L1504211512544B65347F11_PH00.fits’

L1504211512544B6534 100%[===================>]  30.14M  2.12MB/s    in 14s     

2025-09-15 11:12:21 (2.15 MB/s) - ‘L1504211512544B65347F11_PH00.fits’ saved [31608000/31608000]

--2025-09-15 11:12:21--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/L1504211512544B65347F11_PH01.fits
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56321280 (54M) [application/fits]
Saving to: ‘L1504211512544B653

In [2]:
!mkdir data
!mv *SC00.fits spacecraft.fits
!mv *.fits ./data

You'll first need to make a file list with the names of your input event files:

In [3]:
!ls ./data/*PH*.fits > ./data/CenA.list
!cat ./data/CenA.list

./data/L1504211512544B65347F11_PH00.fits
./data/L1504211512544B65347F11_PH01.fits
./data/L1504211512544B65347F11_PH02.fits


In the following analysis we've assumed that you've named your list of data files `CenA.list` and renamed the spacecraft file (`L1504211512544B65347F11_SC00.fits`) to `spacecraft.fits`.

# Perform Event Selections

We are going to follow the prescription described in the unbinned likelihood tutorial to filter and prepare our data set.

First, execute **gtselect**:

In [4]:
%%bash
gtselect evclass=128 evtype=3
    @./data/CenA.list
    ./data/CenA_filtered.fits
    201.47
    -42.97
    10
    239557420
    265507200
    300
    300000
    90

Input FT1 file[@./data/CenA.list]     @./data/CenA.list
Output FT1 file[./data/CenA_filtered.fits]     ./data/CenA_filtered.fits
RA for new search center (degrees) (0:360) [201.47]     201.47
Dec for new search center (degrees) (-90:90) [-42.97]     -42.97
radius of new search region (degrees) (0:180) [10]     10
start time (MET in s) (0:) [239557420]     239557420
end time (MET in s) (0:) [265507200]     265507200
lower energy limit (MeV) (0:) [300]     300
upper energy limit (MeV) (0:) [300000]     300000
maximum zenith angle value (degrees) (0:180) [90]     90
Done.


Next, you need to run **gtmktime** with a similar filter as that used in the publication.

In [5]:
%%bash
gtmktime
    ./data/spacecraft.fits
    (DATA_QUAL>0)&&(LAT_CONFIG==1)
    no
    ./data/CenA_filtered.fits
    ./data/CenA_filtered_gti.fits

Spacecraft data file[./data/spacecraft.fits]     ./data/spacecraft.fits
Filter expression[(DATA_QUAL>0)&&(LAT_CONFIG==1)]     (DATA_QUAL>0)&&(LAT_CONFIG==1)
Apply ROI-based zenith angle cut[no]     no
Event data file[./data/CenA_filtered.fits]     ./data/CenA_filtered.fits
Output event file name[./data/CenA_filtered_gti.fits]     ./data/CenA_filtered_gti.fits


# Create and View a Counts Map

Now we can go ahead and create a counts map of the region so we can visualize the lobes and see the core of CenA in gamma rays.

We can't make any quantitative statements yet, since we haven't produced a template to model but we can at least look and see what we have.

In [6]:
%%bash
gtbin
    CMAP
    ./data/CenA_filtered_gti.fits
    ./data/CenA_CMAP.fits
    NONE
    140
    140
    0.1
    CEL
    201.47
    -42.97
    0
    AIT

This is gtbin version HEAD
Type of output file (CCUBE|CMAP|LC|PHA1|PHA2|HEALPIX) [CMAP]     CMAP
Event data file name[./data/CenA_filtered_gti.fits]     ./data/CenA_filtered_gti.fits
Output file name[./data/CenA_CMAP.fits]     ./data/CenA_CMAP.fits
Spacecraft data file name[NONE]     NONE
Size of the X axis in pixels[140]     140
Size of the Y axis in pixels[140]     140
Image scale (in degrees/pixel)[0.1]     0.1
Coordinate system (CEL - celestial, GAL -galactic) (CEL|GAL) [CEL]     CEL
First coordinate of image center in degrees (RA or galactic l)[201.47]     201.47
Second coordinate of image center in degrees (DEC or galactic b)[-42.97]     -42.97
Rotation angle of image axis, in degrees[0]     0
Projection method e.g. AIT|ARC|CAR|GLS|MER|NCP|SIN|STG|TAN:[AIT]     AIT


gtbin: WARNING: No spacecraft file: EXPOSURE keyword will be set equal to ontime.


Open this FITS image up (`CenA_CMAP.fits`) in your favorite FITS viewer such as *ds9*. If you use the default settings, you'll see the following (using the 'heat' color map):

<img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_CMAP.png'>

Visualizing the lobes with this raw image is not easy. You can kind of see some emission in the center but not really resolve it.

However, you can smooth this image in *ds9* by clicking the 'Analysis Menu' and then selecting 'Smooth'.

This is better, but not quite good enough, so click 'Analysis' again and select 'Smooth Parameters'. This should bring up a dialog box where you can select the Kernel Radius. Set this to 9 and click 'Apply' and then close the dialog box.

The resulting image is pretty washed out now, so click 'Color' and then select 'b'. If you play with the color scale by holding the right mouse button while dragging it around you can wind up with a smoothed count map that looks a little like the one in the publication:

<img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_CMAP_smooth.png'>

We're not going to get something perfect here since we're manually playing with the color scaling and the paper used a sophisticated adaptive smoothing algorithm while we're just using the simple Gaussian algorithm included in *ds9*.

However, you can already see the central core of CenA and some diffuse emission to the north and south of the core that kind of looks like extended emission.

You can also see several point sources in the ROI that are other gamma-ray sources we'll need to model. Since Cen A is near the galactic plane, you can make out some of the galactic diffuse emission at the bottom of the image.

# Create a Template for the Extended Emission

Now that we've convinced ourselves that there is extended lobe emission, we need to find a way to describe the spatial extent.

In this case, we'll follow the example of the publication and use the WMAP k-band observations as a template. The exact WMAP template that was used in the publication is not available but we can get a very similar one from [NASA's SkyView](http://skyview.gsfc.nasa.gov/).

Navigate to SkyView using and click on the [SkyView Query Form](http://skyview.gsfc.nasa.gov/cgi-bin/query.pl) link. We want to download the WMAP data in a region around Cen A, so input `Cen A` into the `Coordinates of Source` box of the form. Then, select `WMAP K` in the `Infrared CMB:` box.

We need to match our ROI, so under `Common Options` input `140` as the `Image size (pixels)` and 14 as the `Image Size (degrees)`. Once you've input all of this, hit the `Submit Request` button.

<img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/SkyView_Query.png'>

On the resulting page, you'll see an image of the WMAP data showing a fuzzy image of Cen A in the K-Band.

This is nice, but what we really want is the FITS image, so click the FITS link and download it into your working directory.

In the following section, we're going to assume that the SkyView image is called `skv.fits`, so rename the downloaded file accordingly.

You can also download the WMAP image directly by running the code cell below.

In [7]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/skv.fits
!mv *.fits ./data

--2025-09-15 11:13:56--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/skv.fits
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89280 (87K) [application/fits]
Saving to: ‘skv.fits’

skv.fits            100%[===================>]  87.19K  --.-KB/s    in 0.002s  

2025-09-15 11:13:57 (37.9 MB/s) - ‘skv.fits’ saved [89280/89280]



Here's what the downloaded WMAP K-band image looks like in ds9 using the 'heat' color map and square-root scaling without any smoothing:

<img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/wmap-k.png'>

The raw image by itself is not in the correct form for the analysis we want to do.

First of all, we want to subtract out the central core of Cen A and we want to divide the map into two parts so that we can separately model the north and south lobes. Furthermore, there are a few things we need to do to make any map compatible with the Fermitools:

* The template must be in J2000 coordinates. The likelihood code does not transpose coordinates into J2000.
* The background must be set to 0. The likelihood code will use any pixel over 0 as part of the template.
* The total flux must be normalized to 1.

The first point is already taken care of since the WMAP image is in J2000 coordinates.

We'll take care of the second point by setting all of the points below a certain value to 0. This isn't the best way to do it, but it's quick, which is what we're going for in this thread.

In the final step, we'll integrate over the entire map to get our normalization factor and divide each pixel by this number.

So let's get started!

We're going to use `pyFits` to do this which is included in the Fermitools (you could use a similar tools like IDL or ftools if you wanted). You can find more details on pyFits at the [pyFits website](http://www.stsci.edu/resources/software_hardware/pyfits).

In [8]:
import astropy.io.fits as pyfits
import numpy as np

wmap_image = pyfits.open("./data/skv.fits")

You can view the header of the image via the following command. The image is in the first HDU of the FITS file so you access the '0'th element of the `wmap_image` object.

In [9]:
print(wmap_image[0].header)

SIMPLE  =                    T / Written by SkyView Tue Jul 12 16:03:27 EDT 2011BITPIX  =                  -32 / 4 byte floating point                          NAXIS   =                    2 / Two dimensional image                          NAXIS1  =                  140 / Width of image                                 NAXIS2  =                  140 / Height of image                                CRVAL1  =   201.36506299999994 / Reference longitude                            CRVAL2  =  -43.019113000000004 / Reference latitude                             RADESYS = 'FK5     '           / Coordinate system                              EQUINOX =               2000.0 / Epoch of the equinox                           CTYPE1  = 'RA---TAN'           / Coordinates -- projection                      CTYPE2  = 'DEC--TAN'           / Coordinates -- projection                      CRPIX1  =                 70.5 / X reference pixel                              CRPIX2  =                 70.5 / Y refer

Now we do a rough background suppression by setting any pixels less than 0.5 mK to 0. We'll also save the resulting image to a file so we can see what we've done.

In [10]:
wmap_image[0].data[wmap_image[0].data < 0.5] = 0.0
wmap_image.writeto('./data/CenA_wmap_k_above5.fits')

If you open up this image in *ds9*, you'll notice that there's still some background left in the south east portion of the image, so we'll get rid of this by setting any pixels that are more than 5 degrees away from the center to 0 and write this out to a file.

Remember that the image is 14 degrees and has a pixel size of 0.1 degrees. We're going to create a 2-d array that's filled with the distance to the center pixel.

In [11]:
x = np.arange(-7,7,0.1)
y = np.arange(-7,7,0.1)
xx, yy = np.meshgrid(x, y, sparse=True)
dist = np.sqrt(xx**2 + yy**2)

Now, set all of the pixels in the wmap data where `dist` is greater than 5 to 0.

In [12]:
wmap_image[0].data[dist > 5] = 0.0
wmap_image.writeto('./data/CenA_wmap_k_nobkgrnd.fits')

The resulting image should only have emission from the radio galaxy left and the rest of the image set to 0.

This is important because any non-zero pixels will be treated as part of the template by the likelihood code and will skew your results.

Now we will continue on to subtract out the center 1 degree of the image so that the core of Cen A is not included in the template.

In [13]:
wmap_image[0].data[dist < 1.0] = 0.0
wmap_image.writeto('./data/CenA_wmap_k_nocenter.fits')

At this point, we want to divide up our map into north and south pieces and normalize each of them. We need to normalize the flux of each template that we're using to 1. This means doing a two dimensional integral over the image making sure to take into account the size of each bin.

In practice, this means summing up all of the bins, multiplying this by (pi/180)^2 and multiplying this by the pixel area. We will now make the north map by setting all the south pixels to zero, normalize it and save it. We'll then close the image, open the image we saved before we did the zeroing out of the south pixels, and do the same for the south region.

In [14]:
wmap_image[0].data[0:69,0:140] = 0
norm = np.sum(wmap_image[0].data) * (np.pi/180)**2 * (0.1**2)
wmap_image[0].data = wmap_image[0].data / norm
wmap_image.writeto('./data/CenA_wmap_k_nocenter_N.fits')
wmap_image.close()

Now open the file back up and do the same for the south region.

In [15]:
wmap_image = pyfits.open('./data/CenA_wmap_k_nocenter.fits')
wmap_image[0].data[70:140,0:140]=0
norm = np.sum(wmap_image[0].data) * (np.pi/180)**2 * (0.1**2)
wmap_image[0].data = wmap_image[0].data / norm
wmap_image.writeto('./data/CenA_wmap_k_nocenter_S.fits')
wmap_image.close()

In the end, you should have two FITS files that look like the following images. The max value in the North template should be around 1670 and the max in the South should be around 900.

If you are far away from these values, make sure you executed the above steps correctly.

In [16]:
from IPython.display import HTML
display(HTML("<table><tr><td><img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/wmap-k-nocenter-S.png'></td><td><img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/wmap-k-nocenter-N.png'></td></tr></table>"))

,


# Create a 3D Counts Map and Compute the Livetime

Now we can continue to follow the binned analysis prescription by creating a 3D counts map.

In [17]:
%%bash
gtbin
    CCUBE
    ./data/CenA_filtered_gti.fits
    ./data/CenA_CCUBE.fits
    NONE
    140
    140
    0.1
    CEL
    201.47
    -42.97
    0
    AIT
    LOG
    300
    300000
    30

This is gtbin version HEAD
Type of output file (CCUBE|CMAP|LC|PHA1|PHA2|HEALPIX) [CMAP]     CCUBE
Event data file name[./data/CenA_filtered_gti.fits]     ./data/CenA_filtered_gti.fits
Output file name[./data/CenA_CMAP.fits]     ./data/CenA_CCUBE.fits
Spacecraft data file name[NONE]     NONE
Size of the X axis in pixels[140]     140
Size of the Y axis in pixels[140]     140
Image scale (in degrees/pixel)[0.1]     0.1
Coordinate system (CEL - celestial, GAL -galactic) (CEL|GAL) [CEL]     CEL
First coordinate of image center in degrees (RA or galactic l)[201.47]     201.47
Second coordinate of image center in degrees (DEC or galactic b)[-42.97]     -42.97
Rotation angle of image axis, in degrees[0]     0
Projection method e.g. AIT|ARC|CAR|GLS|MER|NCP|SIN|STG|TAN:[AIT]     AIT
Algorithm for defining energy bins (FILE|LIN|LOG) [LOG]     LOG
Start value for first energy bin in MeV[100]     300
Stop value for last energy bin in MeV[500000]     300000
Number of logarithmically uniform energy b

gtbin: WARNING: No spacecraft file: EXPOSURE keyword will be set equal to ontime.


Then compute the livetime. Remember that we applied a zenith cut to the data when we performed our event selections. You will need to apply the zenith cut to the exposure by including `zmax=90` as an argument on the command line. This will take something like 30 minutes to complete.

In [18]:
%%bash
gtltcube zmax=90
    ./data/CenA_filtered_gti.fits
    ./data/spacecraft.fits
    ./data/CenA_ltcube.fits
    0.025
    1

Event data file[./data/SwiftJ1644_filtered_gti.fits]     ./data/CenA_filtered_gti.fits
Spacecraft data file[./data/L181102105258F4F0ED2772_SC00.fits]     ./data/spacecraft.fits
Output file[./data/SwiftJ1644_ltCube.fits]     ./data/CenA_ltcube.fits
Step size in cos(theta) (0.:1.) [0.025]     0.025
Pixel size (degrees)[1]     1


Working on file ./data/spacecraft.fits
.....................!


# Generate XML Model File

We have developed a model file (`CenA_model.xml`) based on the 4FGL catalog that works for this ROI and the length of time of the observations. If you want to develop your own, you should look at the other threads where this is done.

Also make sure you have the most recent galactic diffuse and isotropic model files which can be downloaded [from this page](https://fermi.gsfc.nasa.gov/ssc/data/access/lat/BackgroundModels.html). They should also be in your Fermitools installation in the `$(FERMI_DIR)/refdata/fermi/galdiffuse` directory.

Here is a code cell that fetches the needed files:

In [19]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_model.xml

--2025-09-15 11:21:52--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_model.xml
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162278 (158K) [application/xml]
Saving to: ‘CenA_model.xml’

CenA_model.xml      100%[===================>] 158.47K   474KB/s    in 0.3s    

2025-09-15 11:21:52 (474 KB/s) - ‘CenA_model.xml’ saved [162278/162278]



In [20]:
!mv *.xml ./data

If you open `CenA_model.xml` up, you will see background sources, the galactic and extragalactic backgrounds, and the three sources of interest (`CenA`, `CenA_NorthLobe` and `CenA_SouthLobe`). Note that the Lobes reference the spatial models we created in the previous section (`CenA_wmap_k_nocenter_N.fits` and `CenA_wmap_k_nocenter_S.fits`).

For these three sources we are fitting a powerlaw model (see the [Cicerone](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/) for descriptions of the different spectral and spatial models). The core of Cen A will be modeled by a point source (`<spatialModel type="SkyDirFunction">`) and the two lobes will be modeled using the templates we created from the WMAP sky map (`<spatialModel file="some_map.fits" type="SpatialMap">`).

# Compute the Binned Exposure and Source Maps

Now, we need to compute a binned exposure map. Make sure you tell it `none` when asked for a Counts cube so that you can choose the dimensions of the exposure map.
>**Note**: If you get a `File not found` error when using CALDB to select your IRF, you will need to specify the IRF name. In this case we want to use the `P8R3_SOURCE_V3` IRF.

In [21]:
%%bash
gtexpcube2
    ./data/CenA_ltcube.fits
    none
    ./data/CenA_BinnedExpMap.fits
    P8R3_SOURCE_V3
    400
    400
    0.1
    201.47
    -42.97
    0
    AIT
    CEL
    300
    300000
    30

Livetime cube file[./data/3C279_front_ltcube.fits]     ./data/CenA_ltcube.fits
Counts map file[none]     none
Output file name[./data/3C279_back_BinnedExpMap.fits]     ./data/CenA_BinnedExpMap.fits
Response functions to use[P8R3_SOURCE_V2]     P8R3_SOURCE_V3
Size of the X axis in pixels[1800]     400
Size of the Y axis in pixels[900]     400
Image scale (in degrees/pixel)[0.2]     0.1
First coordinate of image center in degrees (RA or galactic l)[193.98]     201.47
Second coordinate of image center in degrees (DEC or galactic b)[-5.82]     -42.97
Rotation angle of image axis, in degrees[0]     0
Projection method e.g. AIT|ARC|CAR|GLS|MER|NCP|SIN|STG|TAN[AIT]     AIT
Coordinate system (CEL - celestial, GAL -galactic) (CEL|GAL) [CEL]     CEL
Start energy (MeV) of first bin[100]     300
Stop energy (MeV) of last bin[500000]     300000
Number of logarithmically-spaced energy bins[37]     30


Computing binned exposure map....................!


Note that we've matched the energy dimensions of our binned exposure map to our counts cube but we've made it 40 degrees square instead of 14. This is so that **gtsrcmaps** can calculate the exposure on sources not in our ROI that still might contribute to emission within our ROI.

To achieve this, **gtsrcmaps** finds the farthest point from the center of our ROI (i.e. one of the corners, a distance of 7$\cdot$sqrt(2) degrees from the center) adds 10 degrees to this and then convolves the model counts map that it creates with the PSF.

Since the PSF at low energies (100s of MeV) is on the order of 4 - 10 degrees in radius, we need to add an additional 10 degrees to account for the convolution. The total distance from the center in our case is thus 7$\cdot$sqrt(2) + 10 + (4-10) or approximately 24 - 30 degrees. This means the smallest square region that can contain this circle is approximately 34 - 43 degrees on a side ((radius/sqrt(2))$\cdot$2)).

Don't worry about getting it exactly right; **gtscrmaps** will fail with an error about 'emapbnds' if you input an exposure cube that's too small, and you can go back and make it larger.

Now run **gtsrcmaps** to generate a source map using the model file we just created.

**Note**: Make sure that, in the xml file (`CenA_model.xml`), you specify where your diffuse source files (`gll_iem_v07.fits` and `iso_P8R3_SOURCE_V3_v1.txt`) are. In most cases, this is `$FERMI_DIR/refdata/fermi/galdiffuse`.

In [23]:
%%bash
gtsrcmaps
    ./data/CenA_ltcube.fits
    ./data/CenA_CCUBE.fits
    ./data/CenA_model.xml
    ./data/CenA_BinnedExpMap.fits
    ./data/CenA_srcMaps.fits
    CALDB

Exposure hypercube file[./data/CenA_ltcube.fits]     ./data/CenA_ltcube.fits
Counts map file[./data/CenA_CCUBE.fits]     ./data/CenA_CCUBE.fits
Source model file[./data/CenA_model.xml]     ./data/CenA_model.xml
Binned exposure map[./data/CenA_BinnedExpMap.fits]     ./data/CenA_BinnedExpMap.fits
Source maps output file[./data/CenA_srcMaps.fits]     ./data/CenA_srcMaps.fits
Response functions[CALDB]     CALDB


Generating SourceMap for 4FGL J1147.0-3812 31....................!
Generating SourceMap for 4FGL J1149.5-4029 31....................!
Generating SourceMap for 4FGL J1150.6-4823 31....................!
Generating SourceMap for 4FGL J1150.7-4613 31....................!
Generating SourceMap for 4FGL J1155.5-3418 31....................!
Generating SourceMap for 4FGL J1202.9-5717 31....................!
Generating SourceMap for 4FGL J1203.4-3925 31....................!
Generating SourceMap for 4FGL J1204.5-5032 31....................!
Generating SourceMap for 4FGL J1207.4-4536 31....................!
Generating SourceMap for 4FGL J1207.4-5050 31....................!
Generating SourceMap for 4FGL J1208.5-5512 31....................!
Generating SourceMap for 4FGL J1209.0-4630 31....................!
Generating SourceMap for 4FGL J1211.0-3800 31....................!
Generating SourceMap for 4FGL J1213.8-4345 31....................!
Generating SourceMap for 4FGL J1213.9-4416 31.................

# Run the Likelihood Analysis

It's time to actually run the likelihood analysis now.

First, you need to import the pyLikelihood module and then the BinnedAnalysis functions.

As discussed in the likelihood thread, it's best to do a two-pass fit here: the first with the MINUIT optimizer and a looser tolerance, and then finally with the NewMinuit optimizer and a tighter tolerance. This allows us to zero in on the best fit parameters.

For more details on the pyLikelihood module, check out the [pyLikelihood Usage Notes](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/python_usage_notes.html).

In [24]:
import pyLikelihood

from BinnedAnalysis import*
obs = BinnedObs(srcMaps='./data/CenA_srcMaps.fits',expCube='./data/CenA_ltcube.fits',binnedExpMap='./data/CenA_BinnedExpMap.fits',irfs='P8R3_SOURCE_V3')
like1 = BinnedAnalysis(obs,'./data/CenA_model.xml',optimizer='MINUIT')

Set the tolerance to something large to start out with, do the first fit and save the results to a file.

Here, we get the minimization object (like1obj) from the logLike object so that we can access it later. We pass this object to the fit routine so that it knows which fitting object to use.

In [25]:
like1.tol = 0.1
like1obj = pyLike.Minuit(like1.logLike)
like1.fit(verbosity=0,covar=True,optObject=like1obj)
like1.logLike.writeXml('./data/CenA_fit1.xml')

Let's check how the convergence of this first fit went.

In [26]:
like1obj.getQuality()

3

This output corresponds to the `MINUIT` fit quality.

A "good" fit corresponds to a value of `fit quality = 3`; if you get a lower value, it is likely that there is a problem with the error matrix. According to the Minuit documentation possible values for "fit quality" are:

* 0 - Error matrix not calculated at all
* 1 - Diagonal approximation only, not accurate
* 2 - Full matrix, but forced positive-definite (i.e. not accurate)
* 3 - Full accurate covariance matrix (After MIGRAD, this is the indication of normal convergence.)

Now, create a new BinnedAnalysis object using the same BinnedObs object from before and the results of the previous fit and tell it to use the NewMinuit optimizer. We also double check that the tolerance is what we want (1e-8) and then we fit the model and save the results.

In [27]:
like2 = BinnedAnalysis(obs,'./data/CenA_fit1.xml',optimizer='NewMinuit')
like2.tol = 1e-8
like2obj = pyLike.NewMinuit(like2.logLike)
like2.fit(verbosity=0,covar=True,optObject=like2obj)

80025.65565894674

We can check to see if NewMinuit converged like this:

In [28]:
print(like2obj.getRetCode())

0


Great, it converged this time, so let's save the fitted model. The return codes for NewMinuit are different than those for Minuit (it's a bit mask).

The only thing you really need to know is that if this number is anything but `0`, the fit didn't converge and you have to keep trying.

In [29]:
like2.logLike.writeXml('./data/CenA_fit2.xml')

Now that we've done the full fit we can verify that we've gotten values close to what's in the publication. First check the results for the core:

In [30]:
like2.Ts('Cen A')
like2.model['Cen A']
like2.flux('Cen A',emin=100,emax=100000)
like2.fluxError('Cen A',emin=100,emax=100000)

2.249417294934077e-08

In [31]:
help(like2)

Help on BinnedAnalysis in module BinnedAnalysis object:

class BinnedAnalysis(AnalysisBase.AnalysisBase)
 |  BinnedAnalysis(binnedData, srcModel=None, optimizer='Drmngb', use_bl2=False, verbosity=0, psfcorr=True, wmap=None, config=None, delete_local_fixed=False)
 |  
 |  Method resolution order:
 |      BinnedAnalysis
 |      AnalysisBase.AnalysisBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, binnedData, srcModel=None, optimizer='Drmngb', use_bl2=False, verbosity=0, psfcorr=True, wmap=None, config=None, delete_local_fixed=False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __setitem__(self, name, value)
 |  
 |  addSource(self, src, binnedConfig=None)
 |      Add a source to the active model.  You should pass a source
 |      object to this function.  Example:
 |      
 |      > test_src = pyLike.PointSource(0, 0, like.observation.observation)
 |      > pl = pyLike.SourceFactory_funcFactory().create("PowerLaw")
 | 

So, the core is detected with high significance (TS = 325) and the fit resulted in a spectral index of $2.64 ± 0.1$. The integral flux above 100 MeV is $(1.3 ± 0.2) \cdot 10^{-7}$. This matches well (within statistical errors) the index and flux reported in the article (index = 2.67 ± 0.10, flux = 1.50 +0.25/-0.22).

In [32]:
like2.Ts('CenA_NorthLobe')
like2.model['CenA_NorthLobe']
like2.flux('CenA_NorthLobe',emin=100,emax=100000)
like2.fluxError('CenA_NorthLobe',emin=100,emax=100000)
like2.Ts('CenA_SouthLobe')
like2.model['CenA_SouthLobe']
like2.flux('CenA_SouthLobe',emin=100,emax=100000)
like2.fluxError('CenA_SouthLobe',emin=100,emax=100000)

3.37132153611994e-08

The north lobe is marginally detected with a low TS value while the south lobe is detected at a higher TS.


The main reasons for any discrepancy with the original paper is that we are using newly reconstructed data, different IRFs, different background models, a slightly different template than was used by the LAT team, and a different model file based on the 4FGL catalog.

The fact that our results are still close indicates that the method is robust. If you get results that are outside of the errors of the thread, double check that you've followed the thread accurately.

# Create some Residual Maps

It's easiest to visualize our results by generating some model maps based on the model we just created and then use these models to create residuals maps based on the actual counts map. We're going to create four different model maps:

* `CenA_ModelMap_All.fits`: A map of the full model (nothing commented out)
* `CenA_ModelMap_AllBkgrnd.fits`: A map of all of the background sources (comment out the Cen A sources)
* `CenA_ModelMap_CenA.fits`: A map of just the Cen A sources (comment out everything but the Cen A sources)
* `CenA_ModelMap_Diff.fits`: A map of just the diffuse background sources (comment out everything but the isotropic and galactic sources)

To do this, we'll need to edit the output xml model file (`CenA_fit2.xml`) each time and then run **gtmodel**.

In [37]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_fit2.xml
!mv *.xml ./data

--2025-09-15 11:43:47--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_fit2.xml
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133774 (131K) [application/xml]
Saving to: ‘CenA_fit2.xml’

CenA_fit2.xml       100%[===================>] 130.64K   753KB/s    in 0.2s    

2025-09-15 11:43:47 (753 KB/s) - ‘CenA_fit2.xml’ saved [133774/133774]



Again, **make sure that the xml points to the correct location for the diffuse source files**.

Other than properly specifying the location of the diffuse sources, you don't need to edit the XML for the first one:

In [46]:
%%bash
gtmodel
    ./data/CenA_srcMaps.fits
    ./data/CenA_fit2.xml
    ./data/CenA_ModelMap_All.fits
    CALDB
    ./data/CenA_ltcube.fits
    ./data/CenA_BinnedExpMap.fits

Source maps (or counts map) file[./data/CenA_srcMaps.fits]     ./data/CenA_srcMaps.fits
Source model file[./data/CenA_fit2.xml]     ./data/CenA_fit2.xml
Output file[./data/CenA_ModelMap_All.fits]     ./data/CenA_ModelMap_All.fits
Response functions[CALDB]     CALDB
Exposure cube[./data/CenA_ltcube.fits]     ./data/CenA_ltcube.fits
Binned exposure map[./data/CenA_BinnedExpMap.fits]     ./data/CenA_BinnedExpMap.fits


For the next file, comment out the Cen A sources in the xml file (you comment out text in xml using `<!--` and `-->` before and after the text you want to comment out) and run **gtmodel** again.

```xml
<!-- <source name="Cen A" type="PointSource">
    <spectrum type="PowerLaw">
      <parameter error="0.1143998413" free="1" max="10000" min="0.0001" name="Prefactor" scale="1e-11" value="1.354990763" />
      <parameter error="0.09903003339" free="1" max="10" min="0" name="Index" scale="-1" value="2.636336026" />

.
.
.
.
.

    <spatialModel file="CenA_wmap_k_nocenter_S.fits" map_based_integral="true" type="SpatialMap">
      <parameter free="0" max="1000" min="0.001" name="Prefactor" scale="1" value="1" />
    </spatialModel>
  </source> -->
```

This generates a model map without the CenA sources, leaving the background intact.

In [47]:
%%bash
gtmodel
    ./data/CenA_srcMaps.fits
    ./data/CenA_fit2.xml
    ./data/CenA_ModelMap_All.fits
    CALDB
    ./data/CenA_ltcube.fits
    ./data/CenA_BinnedExpMap.fits

Source maps (or counts map) file[./data/CenA_srcMaps.fits]     ./data/CenA_srcMaps.fits
Source model file[./data/CenA_fit2.xml]     ./data/CenA_fit2.xml
Output file[./data/CenA_ModelMap_All.fits]     ./data/CenA_ModelMap_All.fits
Response functions[CALDB]     CALDB
Exposure cube[./data/CenA_ltcube.fits]     ./data/CenA_ltcube.fits
Binned exposure map[./data/CenA_BinnedExpMap.fits]     ./data/CenA_BinnedExpMap.fits


Continue to comment out select parts of the model xml file according to the list above and run **gtmodel** afterwards to get the next two files.

`CenA_ModelMap_CenA.fits`: Comment out everything *but* the CenA sources in `CenA_fit2.xml`.

In [48]:
%%bash
gtmodel
    ./data/CenA_srcMaps.fits
    ./data/CenA_fit2.xml
    ./data/CenA_ModelMap_All.fits
    CALDB
    ./data/CenA_ltcube.fits
    ./data/CenA_BinnedExpMap.fits

Source maps (or counts map) file[./data/CenA_srcMaps.fits]     ./data/CenA_srcMaps.fits
Source model file[./data/CenA_fit2.xml]     ./data/CenA_fit2.xml
Output file[./data/CenA_ModelMap_All.fits]     ./data/CenA_ModelMap_All.fits
Response functions[CALDB]     CALDB
Exposure cube[./data/CenA_ltcube.fits]     ./data/CenA_ltcube.fits
Binned exposure map[./data/CenA_BinnedExpMap.fits]     ./data/CenA_BinnedExpMap.fits


`CenA_ModelMap_Diff.fits`: Comment out everything *but* the isotropic and galactic sources.

In [49]:
%%bash
gtmodel
    ./data/CenA_srcMaps.fits
    ./data/CenA_fit2.xml
    ./data/CenA_ModelMap_All.fits
    CALDB
    ./data/CenA_ltcube.fits
    ./data/CenA_BinnedExpMap.fits

Source maps (or counts map) file[./data/CenA_srcMaps.fits]     ./data/CenA_srcMaps.fits
Source model file[./data/CenA_fit2.xml]     ./data/CenA_fit2.xml
Output file[./data/CenA_ModelMap_All.fits]     ./data/CenA_ModelMap_All.fits
Response functions[CALDB]     CALDB
Exposure cube[./data/CenA_ltcube.fits]     ./data/CenA_ltcube.fits
Binned exposure map[./data/CenA_BinnedExpMap.fits]     ./data/CenA_BinnedExpMap.fits


You should end up with the four images below:

In [50]:
from IPython.display import HTML
display(HTML("<table><tr><td><img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_ModelMap_All.png'></td><td><img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_ModelMap_AllBkgrnd.png'></td></tr></table>"))
display(HTML("<table><tr><td><img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_ModelMap_CenA.png'></td><td><img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_ModelMap_Diffuse.png'></td></tr></table>"))

,


,


Now, we'll use the ftool **farith** to subtract these model maps from our data to create residual maps.

In [51]:
display(HTML("<table><tr><td><img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_CMAP_Resid.png'><center>CenA_CMAP_Resid.png: Counts map minus all of the sources in the model showing that there's not really anything left after we subtract everything out.</center></td><td><img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_CMAP_CenA.png'><center>CenA_CMAP_CenA.png: Everything subtracted except the Cen A core and lobes. You can really see the extent of the radio galaxy in this map.</center</td></tr></table>"))
display(HTML("<table><tr><td><img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_CMAP_Bkgrnd.png'><center>CenA_CMAP_Bkgrnd.png: This shows a counts map with the Cen A core and lobes removed. The galactic and extragalactic emission dominates here but you can see a few individual point sources.</center></td><td><img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extended/CenA_CMAP_Sources.png'><center>CenA_CMAP_Sources.png: This shows the region with the galactic and extragalactic emission removed so you can see the point sources along with the Cen A sources.</center></td></tr></table>"))

CenA_CMAP_Resid.png: Counts map minus all of the sources in the model showing that there's not really anything left after we subtract everything out.,CenA_CMAP_CenA.png: Everything subtracted except the Cen A core and lobes. You can really see the extent of the radio galaxy in this map.</center


CenA_CMAP_Bkgrnd.png: This shows a counts map with the Cen A core and lobes removed. The galactic and extragalactic emission dominates here but you can see a few individual point sources.,CenA_CMAP_Sources.png: This shows the region with the galactic and extragalactic emission removed so you can see the point sources along with the Cen A sources.


# Finished!

So, it looks like we've generally reproduced the results from the Cen A paper.

You can go on from here and produce a proper spectrum using the model you have produced here (see the python likelihood thread) and you can plot a profile if you like (try using pyfits and matplotlib, both included in the Fermitools).

You can also create any type of template you wish now to analyze extended diffuse sources using arbitrary shapes (create them in python and use pyfits to save them). Just make sure you follow the prescription outlined above.